In [273]:
import gmsh
gmsh.initialize()
using LinearAlgebra, SparseArrays

In [274]:
# Adatok
height = 100
base = 100
elemSize = 20

approxOrder = 2
internalNodes = 1
quadElements = 0

0

In [275]:
# Geometria megrajzolása
gmsh.model.add("rectangle")

p1 = gmsh.model.occ.addPoint(0, 0, 0)
p2 = gmsh.model.occ.addPoint(base, 0, 0)
p23 = gmsh.model.occ.addPoint(base, height / 2, 0)
p3 = gmsh.model.occ.addPoint(base, height, 0)
p4 = gmsh.model.occ.addPoint(0, height, 0)

l1 = gmsh.model.occ.addLine(p1, p2)
l2 = gmsh.model.occ.addLine(p2, p23)
l23 = gmsh.model.occ.addLine(p23, p3)
l3 = gmsh.model.occ.addLine(p3, p4)
l4 = gmsh.model.occ.addLine(p4, p1)

cl1 = gmsh.model.occ.addCurveLoop([l1, l2, l23, l3, l4])

l5 = gmsh.model.occ.addCircle(base / 2, height / 2, 0, min(base, height) / 4)
cl2 = gmsh.model.occ.addCurveLoop([l5])

sf1 = gmsh.model.occ.addPlaneSurface([cl1, cl2])

#p5 = gmsh.model.occ.addPoint(0, height / 2, 0)
#p6 = gmsh.model.occ.addPoint(base, height / 2, 0)
#l5 = gmsh.model.occ.addLine(p5, p6)

gmsh.model.occ.synchronize()

In [276]:
fixUX = []

function fix_ux(groupOfLines, name="FixUX", grp=fixUX)
    supp = gmsh.model.addPhysicalGroup(1, groupOfLines)
    gmsh.model.setPhysicalName(1, supp, name)
    push!(grp, supp)
end

fixUY = []

function fix_uy(groupOfLines, name="FixUY", grp=fixUY)
    supp = gmsh.model.addPhysicalGroup(1, groupOfLines)
    gmsh.model.setPhysicalName(1, supp, name)
    push!(grp, supp)
end

FXY = []

function f_xy(groupOfLines, fx, fy, name="traction", grp=FXY)
    trac = gmsh.model.addPhysicalGroup(1, groupOfLines)
    gmsh.model.setPhysicalName(1, trac, name)
    push!(grp, [trac, fx, fy])
end

f_xy (generic function with 3 methods)

In [277]:
# megfogások
fix_ux([l1])
fix_uy([l1])

# terhelések
f_xy([l23], 1, 0)

1-element Vector{Any}:
 [3, 1, 0]

In [278]:
# Végeselemes felosztás elvégzése

# lekérjük az összes csomópontot
all = gmsh.model.getEntities(0)
# megadjuk, hogy a csomóponthoz rendelt eleméret mekkora legyen
gmsh.model.mesh.setSize(all, elemSize)
# kiválasztjuk a 8-as számú hálózó algoritmust a 2D-s sf1 felülethez
gmsh.model.mesh.setAlgorithm(2, sf1, 6)
# legeneráljuk a hálót a felület kontúrjához (1D-s)
gmsh.model.mesh.generate(1)
# legeneráljuk a hálót a felülethez (2D-s)
gmsh.model.mesh.generate(2)
# a legenerált háromszög elemeket négyszög elemekké alakítjuk
if quadElements == 1
    gmsh.model.mesh.recombine()
end
# másodfokú elemekhez:
# belső csomópontok használata
if internalNodes == 1
    gmsh.option.setNumber("Mesh.SecondOrderIncomplete", 0) # 0:belső csomópontokkal 1:csak éleken lévő csomópontokkal
else
    gmsh.option.setNumber("Mesh.SecondOrderIncomplete", 1) # 0:belső csomópontokkal 1:csak éleken lévő csomópontokkal
end
# közelítés fokszáma (1-től 5-ig)
gmsh.model.mesh.setOrder(approxOrder)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 50%] Meshing curve 4 (Line)
Info    : [ 70%] Meshing curve 5 (Line)
Info    : [ 90%] Meshing curve 6 (Circle)
Info    : Done meshing 1D (Wall 0.000667177s, CPU 0.000666s)
Info    : 29 nodes 35 elements
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00970952s, CPU 0.008579s)
Info    : 46 nodes 98 elements
Info    : Meshing order 2 (curvilinear on)...
Info    : [  0%] Meshing curve 1 order 2
Info    : [ 20%] Meshing curve 2 order 2
Info    : [ 30%] Meshing curve 3 order 2
Info    : [ 50%] Meshing curve 4 order 2
Info    : [ 60%] Meshing curve 5 order 2
Info    : [ 80%] Meshing curve 6 order 2
Info    : [ 90%] Meshing surface 1 order 2
Info    : Surface mesh: worst distortion = 0.562192 (0 elements in ]0, 0.2]); worst gamma = 0.856124
Info    : Done meshing order 2 (Wall 

In [279]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20221221
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.6.1
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [280]:
# Néhány láthatósági beállítás 0:kikapcs. 1:bekapcs.
gmsh.option.setNumber("Geometry.CurveLabels", 0)
gmsh.option.setNumber("Mesh.Points", 0)
gmsh.option.setNumber("Mesh.Lines", 0)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("Mesh.NodeLabels", 0)
gmsh.option.setNumber("Mesh.LineLabels", 0)
gmsh.option.setNumber("Mesh.SurfaceLabels", 0)

In [281]:
# Anyagállandók mátrixa SA feladathoz
E = 2e5
ν = 0.3
D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν 0; ν 1-ν 0; 0 0 (1-2ν)/2]

3×3 Matrix{Float64}:
 2.69231e5  1.15385e5      0.0
 1.15385e5  2.69231e5      0.0
 0.0        0.0        76923.1

In [282]:
# Anyagállandók mátrixa ÁSF feladathoz
E = 2e5
ν = 0.3
D = E / (1 - ν^2) * [1 ν 0; ν 1 0; 0 0 (1-ν)/2]

3×3 Matrix{Float64}:
     2.1978e5  65934.1           0.0
 65934.1           2.1978e5      0.0
     0.0           0.0       76923.1

In [283]:
# Merevségi mátrix felépítése
# csomópontok sorszámának lekérése
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
# végeselemek típusának, sorszámának és kapcsolati mátrixának (connectivity matrix) lekérése
elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(2, -1)
# a lefoglalandó memória méretének becslése
lengthOfIJV = sum([(div(length(elemNodeTags[i]), length(elemTags[i])) * 2)^2 * length(elemTags[i]) for i in 1:length(elemTags)])
nn = []
I = []
J = []
V = []
V = convert(Vector{Float64}, V)
sizehint!(I, lengthOfIJV)
sizehint!(J, lengthOfIJV)
sizehint!(V, lengthOfIJV)
for i in 1:length(elemTypes)
    et = elemTypes[i]
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
    intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2approxOrder))
    numIntPoints = length(intWeights)
    comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
    ∇h = reshape(dfun, :, numIntPoints)
    nnet = zeros(Int, length(elemTags[i]), numNodes)
    invJac = zeros(3, 3numIntPoints)
    Iidx = zeros(Int, numNodes * 2, numNodes * 2)
    Jidx = zeros(Int, numNodes * 2, numNodes * 2)
    for k in 1:numNodes*2, l in 1:numNodes*2
        Iidx[k, l] = l
        Jidx[k, l] = k
    end
    ∂h = zeros(3, numNodes * numIntPoints) # ∂h-t mindig csak felül kellene írni, nem kell újra és újra memóriát foglalni neki.
    B = zeros(3 * numIntPoints, 2 * numNodes) # B-t mindig csak felül kellene írni?
    K1 = zeros(2 * numNodes, 2 * numNodes)
    nn2 = zeros(Int, 2 * numNodes)
    for j in 1:length(elemTags[i])
        elem = elemTags[i][j]
        for k in 1:numNodes
            nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
        end
        jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
        Jac = reshape(jac, 3, :)
        for k in 1:numIntPoints
            invJac[1:3, 3*k-2:3*k] = inv(Jac[1:3, 3*k-2:3*k])'
        end
        ∂h *= 0
        for k in 1:numIntPoints, l in 1:numNodes
            ∂h[1:2, (k-1)*numNodes+l] = invJac[1:2, k*3-2:k*3-1] * ∇h[l*3-2:l*3-1, k] #??????????????????
        end
        B *= 0
        for k in 1:numIntPoints, l in 1:numNodes
            B[k*3-0, l*2-0] = B[k*3-2, l*2-1] = ∂h[1, (k-1)*numNodes+l]
            B[k*3-0, l*2-1] = B[k*3-1, l*2-0] = ∂h[2, (k-1)*numNodes+l]
        end
        K1 *= 0
        for k in 1:numIntPoints
            B1 = B[k*3-2:k*3, 1:2*numNodes]
            K1 += B1' * D * B1 * jacDet[k] * intWeights[k]
        end
        nn2[1:2:2*numNodes] = 2 * nnet[j, 1:numNodes] .- 1
        nn2[2:2:2*numNodes] = 2 * nnet[j, 1:numNodes]
        append!(I, nn2[Iidx[:]])
        append!(J, nn2[Jidx[:]])
        append!(V, K1[:])
    end
    push!(nn, nnet)
end
K = sparse(I, J, V)
K0 = copy(K)
display(K)
#collect(K)

310×310 SparseMatrixCSC{Float64, Int64} with 6260 stored entries:
⎡⠕⣥⣧⡲⠢⠒⠲⠠⠀⠈⠈⠁⠀⠀⠀⢠⢀⠺⠖⡀⠀⠀⠀⠀⠀⢀⢀⠀⠠⠖⠠⠨⢥⣄⡀⠐⠆⣀⠂⠐⎤
⎢⢩⡻⠕⢅⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡐⠨⡈⡭⠂⣀⠀⠀⠀⠀⠀⠀⢀⡀⡭⠀⠈⠑⠪⠅⢀⡀⠒⣀⠁⎥
⎢⢨⠂⠀⠘⠑⣤⣄⠒⠐⠒⠂⠄⠠⠄⠠⡅⠀⣵⣦⣠⠄⠀⠀⠀⠄⠀⠀⠈⣥⣦⠐⠒⠀⠀⠀⢈⡆⠀⡁⠤⎥
⎢⠘⡂⠀⠀⢠⠙⢻⣶⣦⠀⠀⠀⠀⠀⠀⠁⡆⢻⡿⠀⠀⠀⠀⠀⠀⠄⠆⠀⢙⣫⣤⡄⠀⣶⢠⠘⠱⠆⠓⠀⎥
⎢⡀⠀⠀⠀⢰⠀⠈⠛⢻⣶⣦⡀⠀⠀⠀⠰⢣⢰⣮⡄⠀⠀⠀⠀⢠⠀⠄⠀⠀⠀⣶⣶⡶⠉⠘⡄⢨⢡⠀⠀⎥
⎢⠆⠀⠀⠀⠈⠄⠀⠀⠈⠻⠻⣦⡀⣤⠀⢌⡔⠼⢹⠃⠤⣤⣤⡀⠄⡄⣄⠄⠀⠀⠈⠹⠏⠀⠀⠂⠘⠘⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠆⠀⠀⠀⠀⠀⣬⡛⢌⠓⢚⡜⠖⠈⠓⠎⣑⠦⡙⠎⠐⠑⠑⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⣀⢀⠠⠄⠦⠄⠀⢀⡀⡀⢄⣹⢀⡑⢌⣼⢶⣤⡴⠬⣑⠦⣈⢂⢀⣀⡤⠤⢀⣀⣀⣀⡀⠀⠠⢄⡀⠠⠄⎥
⎢⣠⡐⡂⠢⢄⣤⣬⣉⢉⣒⣐⡍⢲⠍⢲⣟⣑⣼⡛⣶⡾⠛⠋⠑⡔⠑⢉⡓⣬⢙⣓⣁⣂⡙⢊⣢⠈⠑⡌⢢⎥
⎢⠘⠡⠣⠋⠈⣻⠛⠋⠊⠿⠷⠒⢦⠀⢀⡿⢻⣬⠻⣦⡴⠶⡔⠦⡰⠢⠮⢙⣼⠛⠉⠉⠾⠞⠟⢦⠳⠶⠈⠃⎥
⎢⠀⠀⠀⠘⠀⠁⠀⠀⠀⠀⠀⣧⢎⢡⢆⢣⣾⠋⢰⡏⠻⣦⡓⡠⠱⣄⢀⠎⠃⠀⠀⠀⠀⠀⠀⠘⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣌⠣⡈⢣⢏⠀⠰⡍⠙⡨⠛⢄⠠⢊⢦⡋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⢀⠀⠀⠀⠁⠀⠄⠀⠒⠀⠥⢊⠁⠈⢐⢔⠉⠰⡊⠑⢦⡠⢂⠑⢄⢆⠀⠀⠀⠀⠀⠀⠀⡠⠂⠀⢔⠀⠈⎥
⎢⠀⠐⠀⢀⡀⠀⠈⠁⠀⠁⠀⠝⢕⠀⠀⡼⢧⠰⣎⢃⡠⠔⡬⠳⠈⠑⠑⢄⡀⠐⠦⠀⠙⠋⠀⠀⡈⠂⢠⡀⎥
⎢⢠⠆⡄⡬⠡⣿⡷⣰⠀⠀⠀⠀⠀⠀⠀⢃⣆⢛⣶⠛⠉⠀⠀⠀⠀⠀⢀⠈⠻⣦⡀⡀⠀⢰⣀⠘⡆⠀⠰⡂⎥
⎢⡀⡂⡀⠀⢰⠀⠀⠿⢸⣿⣆⡀⠀⠀⠀⢸⠝⢸⡇⠀⠀⠀⠀⠀⠀⠀⠈⠃⠀⠨⣿⣿⣫⡂⠨⠃⠀⠀⠀⠀⎥
⎢⠁⢷⡱⡀⠀⠀⢠⣤⡜⠋⠋⠁⠀⠀⠀⠸⣌⠸⣺⠇⠀⠀⠀⠀⠀⠀⡷⠀⢀⣀⠫⠺⠻⣦⡀⠁⢸⠆⢀⡀⎥
⎢⢀⠈⠁⢁⡀⢀⣀⠒⠒⠤⠠⠀⠀⠀⠀⡀⠪⣰⠻⣅⣀⠀⠀⠀⠠⠊⠀⠀⣀⠘⠦⠂⠄⠈⠑⢄⠀⠪⡀⠁⎥
⎢⠈⢡⢠⠈⠈⠉⠱⠆⠆⣒⣒⠀⠀⠀⠀⠱⢆⠀⢹⡆⠀⠀⠀⠀⢀⢄⠢⠈⠈⠉⠀⠀⠲⠖⡠⡀⢱⣶⠈⠁⎥
⎣⢈⠀⠄⠘⠁⡌⠙⠀⠀⠀⠀⠀⠀⠀⠀⠆⠢⣉⠦⠀⠀⠀⠀⠀⡀⠀⠀⠲⠰⠢⠀⠀⠀⠰⠄⠈⠆⠀⡑⢌⎦

In [284]:
# Teher vektor felépítése
dof, dof = size(K)
f = zeros(dof)
for i in 1:length(FXY)
    edge, tracX, tracY = FXY[i]
    trac = [tracX, tracY]
    tags = gmsh.model.getEntitiesForPhysicalGroup(1, edge)
    for i ∈ 1:length(tags)
        elementTypes, elementTags, elemNodeTags = gmsh.model.mesh.getElements(1, tags[i])
        if length(elementTypes) != 1
            error("A peremen nem csak egyfajta végeselem van!")
        end
        elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(elementTypes[1])
        nnoe = reshape(elemNodeTags[1], numNodes, :)'
        intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(elementTypes[1], "Gauss" * string(approxOrder))
        numIntPoints = length(intWeights)
        comp, fun, ori = gmsh.model.mesh.getBasisFunctions(elementTypes[1], intPoints, "Lagrange")
        h = reshape(fun, :, numIntPoints)
        comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(elementTypes[1], intPoints, "GradLagrange")
        ∇h = reshape(dfun, :, numIntPoints)
        H = zeros(2 * numIntPoints, 2 * numNodes)
        for j in 1:numIntPoints
            for k in 1:numNodes
                H[j*2-1, k*2-1] = h[k, j]
                H[j*2-0, k*2-0] = h[k, j]
            end
        end
        f1 = zeros(2 * numNodes)
        nn2 = zeros(Int, 2 * numNodes)
        for l in 1:length(elementTags[1])
            elem = elementTags[1][l]
            jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
            Jac = reshape(jac, 3, :)
            f1 *= 0
            for j in 1:numIntPoints
                H1 = H[j*2-1:j*2, 1:2*numNodes]
                Ja = √((Jac[1, 3*j-2])^2 + (Jac[2, 3*j-2])^2)
                f1 += H1' * trac * Ja * intWeights[j]
            end
            nn2[1:2:2*numNodes] = 2 * nnoe[l, 1:numNodes] .- 1
            nn2[2:2:2*numNodes] = 2 * nnoe[l, 1:numNodes]
            f[nn2] += f1
        end
    end
end
display(f)

310-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 2.777777777777782
 0.0
 2.777777777777784
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [285]:
for i in 1:length(fixUX)
    nodeTags, coord = gmsh.model.mesh.getNodesForPhysicalGroup(1, fixUX[i])
    nodeTags *= 2
    nodeTags .-= 1
    for i ∈ nodeTags
        K[i, :] .= 0
        K[:, i] .= 0
        K[i, i] = 1
        f[i] = 0
    end
    dropzeros!(K)
end
#collect(K)

In [286]:
for i in 1:length(fixUY)
    nodeTags, coord = gmsh.model.mesh.getNodesForPhysicalGroup(1, fixUY[i])
    nodeTags *= 2
    for i ∈ nodeTags
        K[i, :] .= 0
        K[:, i] .= 0
        K[i, i] = 1
        f[i] = 0
    end
    dropzeros!(K)
end
#collect(K)

In [287]:
#=
nodeTags, coord = gmsh.model.mesh.getNodesForPhysicalGroup(1, supp1)
nodeTags *= 2
for i ∈ nodeTags
    K[i, :] .= 0
    K[:, i] .= 0
    K[i, i] = 1
    f[i] = 0
end
dropzeros!(K)
=#

In [288]:
#=
nodeTags, coord = gmsh.model.mesh.getNodesForPhysicalGroup(1, supp1)
nodeTags *= 2
nodeTags .-= 1
for i ∈ nodeTags
    K[i, :] .= 0
    K[:, i] .= 0
    K[i, i] = 1
    f[i] = 0
end
dropzeros!(K)
=#

In [289]:
q = K \ f

310-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.001493416091177376
 -0.0008574277371372919
  0.0028573052241897936
 -0.0006719929199316505
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
  0.0012558855750528337
 -0.0005639301676192684
  0.002502359556638503
 -0.00039048265444644744
  0.0025277338788439463
 -0.00024785416187526286
  0.0001127019017835381
 -1.1532267813679769e-5
  0.0002051468136513548
 -0.0001335096931855444
  0.0014581414668884613
 -0.00036174676817068665

In [290]:
FXY[1][3] / (E / height)

0.0

In [291]:
-(FXY[1][3] / E) * ν * base

-0.0

In [292]:
K0 * q

310-element Vector{Float64}:
  -4.02082939708811
 -13.458513564418235
  -4.465375662323323
  16.691798015559648
   2.777777777777846
   3.078415461263762e-14
   2.7777777777779127
   4.424748089319219e-14
  -1.4177012626148298
  -3.608798706245217
  -5.252190387138512
   1.7882047383139072
  -5.324521486854348
   ⋮
   5.684341886080802e-14
  -4.618527782440651e-14
   3.552713678800501e-14
  -7.815970093361102e-14
   3.9168668308775523e-13
   5.684341886080802e-14
  -5.329070518200751e-15
   3.552713678800501e-15
  -1.4876988529977098e-14
  -4.263256414560601e-14
  -4.231627010591311e-13
  -8.526512829121202e-14

In [293]:
k = base * E / height

200000.0

In [294]:
# Feszültségek számítása
# csomópontok sorszámának lekérése
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
# végeselemek típusának, sorszámának és kapcsolati mátrixának (connectivity matrix) lekérése
elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(2, -1)
numElem = Int[]
σ = Vector{Float64}[]
σx = Vector{Float64}[]
σy = Vector{Float64}[]
σxy = Vector{Float64}[]
for i in 1:length(elemTypes)
    et = elemTypes[i]
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
    s0 = zeros(3numNodes) # csak SA és ÁSF feladatnál, FSZ-nél már 4 kell
    nodeCoord = zeros(numNodes * 3)
    for k in 1:dim, j = 1:numNodes
        nodeCoord[k+(j-1)*3] = localNodeCoord[k+(j-1)*dim]
    end
    comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, nodeCoord, "GradLagrange")
    ∇h = reshape(dfun, :, numNodes)
    nnet = zeros(Int, length(elemTags[i]), numNodes)
    invJac = zeros(3, 3numNodes)
    ∂h = zeros(3, numNodes * numNodes)
    B = zeros(3 * numNodes, 2 * numNodes)
    nn2 = zeros(Int, 2 * numNodes)
    for j in 1:length(elemTags[i])
        elem = elemTags[i][j]
        for k in 1:numNodes
            nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
        end
        jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, nodeCoord)
        Jac = reshape(jac, 3, :)
        for k in 1:numNodes
            invJac[1:3, 3*k-2:3*k] = inv(Jac[1:3, 3*k-2:3*k])'
        end
        ∂h *= 0
        for k in 1:numNodes, l in 1:numNodes
            ∂h[1:2, (k-1)*numNodes+l] = invJac[1:2, k*3-2:k*3-1] * ∇h[l*3-2:l*3-1, k] #??????????????????
        end
        B *= 0
        for k in 1:numNodes, l in 1:numNodes
            B[k*3-0, l*2-0] = B[k*3-2, l*2-1] = ∂h[1, (k-1)*numNodes+l]
            B[k*3-0, l*2-1] = B[k*3-1, l*2-0] = ∂h[2, (k-1)*numNodes+l]
        end
        push!(numElem, elem)
        nn2[1:2:2*numNodes] = 2 * nnet[j, 1:numNodes] .- 1
        nn2[2:2:2*numNodes] = 2 * nnet[j, 1:numNodes]
        s = zeros(9numNodes) # tenzornak 9 eleme van
        sx = zeros(numNodes)
        sy = zeros(numNodes)
        sxy = zeros(numNodes)
        for k in 1:numNodes
            B1 = B[k*3-2:k*3, 1:2*numNodes]
            s0 = D * B1 * q[nn2]
            s[(k-1)*9+1:k*9] = [s0[1], s0[3], 0, s0[3], s0[2], 0, 0, 0, 0]
            sx[k] = s0[1]
            sy[k] = s0[2]
            sxy[k] = s0[3]
        end
        push!(σ, s)
        push!(σx, sx)
        push!(σy, sy)
        push!(σxy, sxy)
    end
end

In [295]:
gmsh.option.setNumber("View.IntervalsType", 3)
gmsh.option.setNumber("View.VectorType", 5)
gmsh.option.setString("View.Format", "%.6g")

In [296]:
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
non = length(nodeTags)
uvec = gmsh.view.add("uvec")
u = zeros(3 * non)
for i in 1:length(nodeTags)
    u[3i-2] = q[2*nodeTags[i]-1]
    u[3i-1] = q[2*nodeTags[i]-0]
end
gmsh.view.addHomogeneousModelData(uvec, 0, "rectangle", "NodeData", nodeTags, u, 0, 3)

gmsh.view.option.setNumber(uvec, "AdaptVisualizationGrid", 1)
gmsh.view.option.setNumber(uvec, "TargetError", -1e-4)
gmsh.view.option.setNumber(uvec, "MaxRecursionLevel", approxOrder + 1)
gmsh.view.option.setNumber(uvec, "Visible", 1)


In [297]:
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
non = length(nodeTags)
ux = gmsh.view.add("ux")
u = zeros(non)
for i in 1:length(nodeTags)
    u[i] = q[2*nodeTags[i]-1]
    #u[3i-1] = q[2*nodeTags[i]-0]
end
gmsh.view.addHomogeneousModelData(ux, 0, "rectangle", "NodeData", nodeTags, u, 0, 1)

gmsh.view.option.setNumber(ux, "AdaptVisualizationGrid", 1)
gmsh.view.option.setNumber(ux, "TargetError", -1e-4)
gmsh.view.option.setNumber(ux, "MaxRecursionLevel", approxOrder + 1)
gmsh.view.option.setNumber(ux, "Visible", 0)


In [298]:
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
non = length(nodeTags)
uy = gmsh.view.add("uy")
u = zeros(non)
for i in 1:length(nodeTags)
    #u[3i-2] = q[2*nodeTags[i]-1]
    u[i] = q[2*nodeTags[i]-0]
end
gmsh.view.addHomogeneousModelData(uy, 0, "rectangle", "NodeData", nodeTags, u, 0, 1)

gmsh.view.option.setNumber(uy, "AdaptVisualizationGrid", 1)
gmsh.view.option.setNumber(uy, "TargetError", -1e-4)
gmsh.view.option.setNumber(uy, "MaxRecursionLevel", approxOrder + 1)
gmsh.view.option.setNumber(uy, "Visible", 0)


In [299]:
S = gmsh.view.add("σ")
gmsh.view.addModelData(S, 0, "rectangle", "ElementNodeData", numElem, σ, 0, 9)

gmsh.plugin.setNumber("Smooth", "View", -1)
gmsh.plugin.run("Smooth")

gmsh.view.option.setNumber(S, "AdaptVisualizationGrid", 1)
gmsh.view.option.setNumber(S, "TargetError", -1e-4)
gmsh.view.option.setNumber(S, "MaxRecursionLevel", approxOrder + 1)
gmsh.view.option.setNumber(S, "Visible", 0)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


In [300]:
Sx = gmsh.view.add("σx")
gmsh.view.addModelData(Sx, 0, "rectangle", "ElementNodeData", numElem, σx, 0, 1)

gmsh.plugin.setNumber("Smooth", "View", -1)
gmsh.plugin.run("Smooth")

gmsh.view.option.setNumber(Sx, "AdaptVisualizationGrid", 1)
gmsh.view.option.setNumber(Sx, "TargetError", -1e-4)
gmsh.view.option.setNumber(Sx, "MaxRecursionLevel", approxOrder + 1)
gmsh.view.option.setNumber(Sx, "Visible", 0)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


In [301]:
Sy = gmsh.view.add("σy")
gmsh.view.addModelData(Sy, 0, "rectangle", "ElementNodeData", numElem, σy, 0, 1)

gmsh.plugin.setNumber("Smooth", "View", -1)
gmsh.plugin.run("Smooth")

gmsh.view.option.setNumber(Sy, "AdaptVisualizationGrid", 1)
gmsh.view.option.setNumber(Sy, "TargetError", -1e-4)
gmsh.view.option.setNumber(Sy, "MaxRecursionLevel", approxOrder + 1)
gmsh.view.option.setNumber(Sy, "Visible", 0)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


In [302]:
Sxy = gmsh.view.add("τxy")
gmsh.view.addModelData(Sxy, 0, "rectangle", "ElementNodeData", numElem, σxy, 0, 1)

gmsh.plugin.setNumber("Smooth", "View", -1)
gmsh.plugin.run("Smooth")

gmsh.view.option.setNumber(Sxy, "AdaptVisualizationGrid", 1)
gmsh.view.option.setNumber(Sxy, "TargetError", -1e-4)
gmsh.view.option.setNumber(Sxy, "MaxRecursionLevel", approxOrder + 1)
gmsh.view.option.setNumber(Sxy, "Visible", 0)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


In [303]:
#xy = gmsh.view.add("2D Plot")
#data = [1.0, 2, 3, 0, 0, 0, 0, 0, 0]
#push!(data, 1, 1, 2)
#gmsh.view.addListData(xy, "SL", 1, data)

In [304]:
#gmsh.write("rect.msh")
#gmsh.write("rect.opt")
#gmsh.write("rect.pos")

In [305]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20221221
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.6.1
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [306]:
gmsh.finalize()